In [1]:
import requests
from bs4 import BeautifulSoup

In [8]:
import re

In [2]:
PAGE_LINK = 'https://www.allmusicals.com/h/hamilton.htm'

In [3]:
page = requests.get(PAGE_LINK)
page.status_code

200

In [4]:
soup = BeautifulSoup(page.text, "html.parser")

In [5]:
lyrics_list = [a['href'] for a in soup.find_all("section", {"class": "lyrics-list"})[0].find_all('a', href=True)]

In [6]:
all_lyrics = ''
for link in lyrics_list:
  temp = requests.get(link)
  lyrics = BeautifulSoup(temp.text, "html.parser").find('div', {'id': "page"})
  all_lyrics += lyrics.text

In [7]:
all_lyrics_clean = all_lyrics.replace('\t', '').replace('\n', '')

In [9]:
#remove brackets who is speaking
all_lyrics_clean = re.sub("[\(\[].*?[\)\]]", "", all_lyrics_clean)

In [10]:
format_title_indices = [m.start() for m in re.finditer('Last Update', all_lyrics_clean)]

In [11]:
substr_to_remove = []
for idx in format_title_indices:
  temp = all_lyrics_clean[idx: idx+100]
  space_idx = temp.find('   ')
  substr_to_remove.append(temp[: space_idx])

In [12]:
#remove dates and titles form text
for s in substr_to_remove:
  all_lyrics_clean = all_lyrics_clean.replace(s, '')

In [13]:
#clean multiple spaces
all_lyrics_clean = re.sub(' +', ' ', all_lyrics_clean)

In [17]:
#clean wrong punctuation
all_lyrics_clean = all_lyrics_clean.replace("?s", "'s").replace("?ll", "'ll")

In [18]:
all_lyrics_list = [line for line in all_lyrics_clean.split('\r') if line!='']

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
import pickle
with open('drive/MyDrive/Colab Notebooks/ham_lyrics_list.pkl', 'wb') as f:
  pickle.dump(all_lyrics_list, f)

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
train, val = train_test_split(all_lyrics_list, test_size = 0.1, shuffle=True)

In [98]:
with open('../../../../drive/MyDrive/Colab Notebooks/train_ham2.txt', 'w') as f:
  f.write("".join(train))

with open('../../../../drive/MyDrive/Colab Notebooks/eval_ham2.txt', 'w') as f:
  f.write("".join(val))

In [24]:
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-razitiaf
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-razitiaf
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.3MB 7.2MB/s 
     |████████████████████████████████| 901kB 34.4MB/s 
  Created wheel for transformers: filename=transformers-4.9.0.dev0-cp37-none-any.whl size=2555688 sha256=713720eaa2d43fde8e21c05adfaa6d1db694cbfa60c49f4569102aec7c88d606
  Stored in directory: /tmp/pip-ephem-wheel-cache-jy75zkqy/wheels/70/d3/52/b3fa4f8b8ef04167ac62e5bb2accb62ae764db2a378247490e
Successfully built transformers


In [25]:
!git clone https://github.com/huggingface/transformers

Cloning into 'transformers'...
remote: Enumerating objects: 77671, done.
remote: Counting objects: 100% (716/716), done.
remote: Compressing objects: 100% (400/400), done.
remote: Total 77671 (delta 427), reused 528 (delta 288), pack-reused 76955
Receiving objects: 100% (77671/77671), 59.87 MiB | 10.44 MiB/s, done.
Resolving deltas: 100% (55374/55374), done.


In [26]:
import os
os.chdir("transformers")
os.chdir("./examples/pytorch/language-modeling")
!ls

README.md	  run_clm_no_trainer.py  run_mlm_no_trainer.py	run_plm.py
requirements.txt  run_clm.py		 run_mlm.py


In [27]:
!pip install -r requirements.txt

     |████████████████████████████████| 266kB 4.3MB/s 
     |████████████████████████████████| 1.2MB 7.5MB/s 
     |████████████████████████████████| 122kB 15.0MB/s 
     |████████████████████████████████| 245kB 17.6MB/s 


In [99]:
!python run_clm.py \
--model_type gpt2 \
--model_name_or_path gpt2 \
--train_file "../../../../drive/MyDrive/Colab Notebooks/train_ham2.txt" \
--do_train \
--validation_file "../../../../drive/MyDrive/Colab Notebooks/eval_ham2.txt" \
--do_eval \
--per_gpu_train_batch_size 1 \
--save_steps -1 \
--num_train_epochs 5 \
--fp16 \
--output_dir="../../../../drive/MyDrive/Colab Notebooks/ham_generator2" \
--overwrite_output_dir True

2021-07-09 09:29:11.758951: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
07/09/2021 09:29:13 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: True
07/09/2021 09:29:13 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=500,
evaluation_strategy=IntervalStrategy.NO,
fp16=True,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
greater_is_better=None,
group_by_length=False,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_colu

In [100]:
from transformers import TFGPT2LMHeadModel
from transformers import GPT2Tokenizer

model = TFGPT2LMHeadModel.from_pretrained("../../../../drive/MyDrive/Colab Notebooks/ham_generator2", from_pt=True)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.9.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'lm_head.weight', 'transformer.h.2.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.4.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

In [101]:
input_ids = tokenizer.encode("wait for it", return_tensors='tf')

In [106]:
#https://huggingface.co/blog/how-to-generate
beam_output = model.generate(
    input_ids, 
    max_length=30, 
    min_length=10, 
    num_return_sequences=5,
    no_repeat_ngram_size=3,
    top_p=0.92,
    temperature=.7,
    do_sample=True,
    top_k=50,
    early_stopping=True
)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


In [107]:
for i, beam in enumerate(beam_output):
  print("{}: {}".format(i,tokenizer.decode(beam)))
  print()

0: wait for it. I would rather be the King of England than be President of France. I am willing to do whatever it takes to make America great

1: wait for it…The King of France, the British, the French, the Dutch, they all made a treaty of war. I am not saying

2: wait for it, but we need to do it right now.

The rest of the world is watching, watching us. It's happening all

3: wait for it!

I know it's crazy, but I'm gonna make it real…I'm gonna tell you what I saw.I

4: wait for it. And I would like to be there. I was just so sorry I didn't get to see you, you know? I'm

